uncomment if executing in google colab

In [1]:
# ! pip uninstall pyro-ppl
# ! pip install pyro-ppl==1.5.1
# ! unzip fonts.zip 
# ! unzip claptchagen.zip
# ! unzip csis
# ! cp csis.py /usr/local/lib/python3.7/dist-packages/pyro/infer

In [2]:
import random
import string
import os
import torch.nn as nn
import torch
import torch.nn.functional as F
import pyro
import numpy as np
import pyro.optim as optim
import pyro.distributions as dist
import pyro.infer
import pyro.optim

from torch.autograd import Variable
from pyro.infer import SVI, Trace_ELBO, TraceGraph_ELBO
from PIL import Image
from claptchagen.claptcha import Claptcha
from torch.distributions import constraints
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
print(pyro.__version__)

captcha_folder = 'generated_captchas'
captchaHeight = 32
captchaWidth = 100
captchaMarginX = 4
captchaMarginY = 4
batch_size = 10

char_dict = string.digits
USE_CUDA = True
MAX_N = 5 # maximum number of letters in a captcha 
MIN_N = 3 # minimum number of letters in a captcha
MIN_NOISE = 0.01 # minimum noise
MAX_NOISE = 0.99 # maximum noise
smoke_test = False
num_steps = 200 if not smoke_test else 10
TrainingSample = 10000 if not smoke_test else 100 # number of captchas generated for training

def randomString():
    """
    return a string with <num_char> random letters
    """
    k = random.randint(MIN_N, MAX_N) # sample number of characters
    
    rndLetters = (random.choice(char_dict) for _ in range(k))
    
    pad_spaces = MAX_N - k # pad the string so the captcha is close to center
    space = " " * (pad_spaces // 2)
    
    return space + "".join(rndLetters) + space

def ramdomNoise():
    """
    return a float between MIN_NOISE, MAX_NOISE
    """
    return random.uniform(MIN_NOISE, MAX_NOISE)

def generate_random_captcha(n, save=False):
    """
    generate n random captchas,
    return a list of texts on the captchas
    """
    # Initialize Claptcha object with random text, FreeMono as font, of size
    # 100x30px, using bicubic resampling filter and adding a bit of white noise
    c = Claptcha(randomString, "fonts/FreeSans.ttf", (captchaWidth, captchaHeight), (captchaMarginX, captchaMarginY),
             resample=Image.BILINEAR, noise=0)
    captcha_generated = [ [] for i in range(MAX_N)]
    for i in range(n):
        c.noise = ramdomNoise()
        if save:
            text, _ = c.write(os.path.join(captcha_folder, 'captcha{}.png'.format(i)))
            os.rename(os.path.join(captcha_folder, 'captcha{}.png'.format(i)),os.path.join(captcha_folder, '{}.png'.format(text + "_" + str(i))))
        text, image = c.image
        text = text.strip()
        image = np.array(image)[:, :, 0] # the generator is gray scale, only keep one channel is enough
        captcha_generated[len(text) - 1].append((text, image, c.noise))
    return captcha_generated
    
captcha_generated = generate_random_captcha(TrainingSample, save=False)
for lst in captcha_generated:
    print("number of samples in group", len(lst))
    # print some sample captcha information generated
    for i, t in enumerate(lst):
        print("text", t[0], "captcha shape", t[1].shape, "noise", t[2])
        if i >= 10:
            break

1.5.1
number of samples in group 0
number of samples in group 0
number of samples in group 3273
text 712 captcha shape (32, 100) noise 0.07700688312384003
text 165 captcha shape (32, 100) noise 0.6417097229132835
text 304 captcha shape (32, 100) noise 0.653944117614349
text 450 captcha shape (32, 100) noise 0.6163655384342778
text 618 captcha shape (32, 100) noise 0.5091439031458551
text 140 captcha shape (32, 100) noise 0.7956781197637947
text 673 captcha shape (32, 100) noise 0.7676395100780321
text 507 captcha shape (32, 100) noise 0.5371319442523937
text 109 captcha shape (32, 100) noise 0.8178013947826746
text 205 captcha shape (32, 100) noise 0.8869671167845189
text 145 captcha shape (32, 100) noise 0.4376002818901202
number of samples in group 3301
text 4098 captcha shape (32, 100) noise 0.6498733748989223
text 7432 captcha shape (32, 100) noise 0.46272872063204656
text 7525 captcha shape (32, 100) noise 0.8370499838880107
text 8989 captcha shape (32, 100) noise 0.09074370774943

In [3]:
def render_image(chars, fonts="fonts/FreeSans.ttf", size=(captchaWidth, captchaHeight), 
                 margin=(captchaMarginX, captchaMarginY), resample=Image.BILINEAR, noise=0.3, use_cuda=False):
    #noise = noise.data.item()
    #print(chars, noise)
    pad_spaces = MAX_N - len(chars)
    space = " " * (pad_spaces // 2)
    chars = space + chars + space
    render = Claptcha(chars, fonts, size, margin, resample=resample, noise=noise)

    
    _ , rendered_image = render.image
    rendered_image = np.array(rendered_image)[:,:,0] # the generator is gray scale, only keep one channel is enough
    rendered_image = np.subtract(np.divide(rendered_image, 255), 0.5)
    rendered_image = torch.from_numpy(rendered_image)
    if use_cuda:
        rendered_image = rendered_image.cuda()
    return rendered_image

In [4]:
class CaptchaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, raw_captchas, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.raw_captchas = raw_captchas
        self.transform = transform

    def __len__(self):
        return len(self.raw_captchas)

    def __getitem__(self, idx):
        label = self.raw_captchas[idx][0]
        image = self.raw_captchas[idx][1]
        noise = self.raw_captchas[idx][2]
        
        image = np.subtract(np.divide(image, 255), 0.5)
        image = torch.from_numpy(image).float()

        if self.transform:
            image = self.transform(image)

        return label, image, noise

In [5]:
def make_loarders(BATCH_SIZE, raw_samples):
    dataloaders = [] # dataloaders for different num of char
    for lst in raw_samples:
        if lst:
            ds = CaptchaDataset(lst)
            dataloader = DataLoader(ds, batch_size=BATCH_SIZE,
                                    shuffle=True, num_workers=0, drop_last=True)
            dataloaders.append(dataloader)
    return dataloaders

def make_batches(dataloaders):
    all_batches = []
    for dl in dataloaders:
        for i_batch, sample in enumerate(dl):
            all_batches.append(sample)
    random.shuffle(all_batches)
    random.shuffle(all_batches)
    return all_batches

TrainLoaders = make_loarders(BATCH_SIZE=batch_size, raw_samples=captcha_generated)

In [6]:
class NoiseNet(nn.Module):

    def __init__(self, img_size, out_size = 1):
        """
        Network for learning noise in a captcha
        """
        super(NoiseNet, self).__init__()
        
        self.img_size = img_size
        self.fc0 = nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1])
        self.fc1 = nn.Linear(img_size[0] * img_size[1], 1024)
        
        self.fc2 = nn.Linear(1024, 1024)
        self.fc20 = nn.Linear(1024, img_size[0] * img_size[1])
        self.fc21 = nn.Linear(img_size[0] * img_size[1], out_size)
        self.softplus = nn.Softplus()
    
    def forward(self, img):
        BS = img.shape[0]
        img = img.reshape(-1, self.img_size[0] * self.img_size[1])
        hidden = F.relu(self.fc0(img))
        hidden = self.fc1(hidden)
        # mean of noise, used in normal distribution
        noise_map = self.fc20(F.relu(self.fc2(F.relu(hidden))))
        mean =  self.fc21(F.relu(noise_map))
        # std used in normal distribution
        sigma = torch.tensor([[1e-8] for _ in range(BS)]).float()
        if USE_CUDA:
            sigma = sigma.cuda()
        return mean, sigma, noise_map

In [8]:
class NumNet(nn.Module):
    def __init__(self, img_size, out_size = 3):
        """
        Network for learning N, number of letters in a captcha
        """
        super(NumNet, self).__init__()
        self.neural_net = nn.Sequential(
            nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1] * 2),
            nn.ReLU(),
            nn.Linear(img_size[0] * img_size[1] * 2, 256),
            nn.ReLU(),
            nn.Linear(256, out_size),
            nn.LogSoftmax(dim=1))
  
    def forward(self, img):
        img = torch.reshape(img, (img.shape[0], img.shape[1] * img.shape[2]))
        prob = self.neural_net(img)
        return prob

In [9]:
class CharNetSingle(nn.Module):
    def __init__(self, img_size, output_size, MAX_N):
        """
        Network for letters in a captcha, given the noise and number of letters
        """
        super(CharNetSingle, self).__init__()
        self.img_size = img_size
        
        # observe layers
        self.nnfc = nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1])
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        
        # branches
        self.pfc1s = nn.ModuleList([nn.Linear(1280, 2048) for i in range(MAX_N)])
        self.pfc2s = nn.ModuleList([nn.Linear(2048, 1024) for i in range(MAX_N)])
        self.pfc3s = nn.ModuleList([nn.Linear(1024, 1024) for i in range(MAX_N)])
        self.pfc4s = nn.ModuleList([nn.Linear(1024, output_size) for i in range(MAX_N)])
        
        self.convBN1 = nn.BatchNorm2d(64)
        self.convBN2 = nn.BatchNorm2d(64)
        self.convBN3 = nn.BatchNorm2d(64)

    def forward(self, img, i, noise_map):
        
        BATCH_SIZE = img.shape[0]
        # feed the noise_map to a linear layer to tune the values
        noise_map = F.relu(self.nnfc(noise_map))
        noise_map = torch.reshape(noise_map, (BATCH_SIZE, 1, self.img_size[0], self.img_size[1]))
    
        img = torch.reshape(img, (BATCH_SIZE, 1, self.img_size[0], self.img_size[1]))
        # compute the difference between original image and noise map
        # i.e. extract the unnoisy image
        img = torch.sub(img, noise_map)

        img = self.pool(F.relu(self.convBN1(self.conv1(img))))
        img = self.pool(F.relu(self.convBN2(self.conv2(img))))
        img = self.pool(F.relu(self.convBN3(self.conv3(img))))
        
        img = torch.reshape(img, (BATCH_SIZE, 1280))
        
        # i is the index of character, use different layers for different indicies
        out = F.relu(self.pfc1s[i](img))
        out = F.relu(self.pfc2s[i](out))
        out = F.relu(self.pfc3s[i](out))
        out = self.pfc4s[i](out)
        out = F.log_softmax(out, dim=1)
        
        return out

In [10]:
def inference(t, use_cuda=False):
    """
    one epoch of inference (iterate the training set once)
    """
    loss = 0
    length = TrainingSample
    loss_group = []
    all_batches = make_batches(TrainLoaders)
    for i_batch, sample_batched in enumerate(all_batches):
        
        img = sample_batched[1]
        if use_cuda:
            img = img.cuda()
        IMG = {"captcha" : img}
        imme_loss = csis.step(observations=IMG)
        loss += imme_loss / length

    print("loss at epoch {} is {}".format(t, loss))
    return loss

In [11]:
def test(n = 0, use_train=False, verbose=False, use_cuda=False):
    """
    benchmarking performance on customized or training set
    """
    if use_train:
        TestLoaders = make_loarders(BATCH_SIZE=1, raw_samples=captcha_generated)
    else:
        test_captcha_generated = generate_random_captcha(n, save=False)
        TestLoaders = make_loarders(BATCH_SIZE=1, raw_samples=test_captcha_generated)
    
    total_correct = 0
    char_correct = 0
    total_char = 0
    all_batches = make_batches(TestLoaders)
    noise_difference = 0
    for i_batch, t in enumerate(all_batches):

        label = t[0][0]
        gt_noise = t[2][0]
        img = t[1]

        if use_cuda:
            img = img.cuda()
        
        IMG = {"captcha" : img}
        
        posterior = csis.run(observations=IMG)
        marginal_num = pyro.infer.EmpiricalMarginal(posterior, "num_char")
        marginal_noise = pyro.infer.EmpiricalMarginal(posterior, "noise")
        with torch.no_grad():

            N_index = marginal_num()
            N = N_index + captchaModel.num_char_domain[0]
            noise = captchaModel._map_to_noise_range(marginal_noise()[0])
            sampled_chars = []
            
            # sample characters one by one
            for i in range(N):
                marginal_char = pyro.infer.EmpiricalMarginal(posterior, "char_{}".format(i))()[0]
                if use_cuda:
                    marginal_char.cpu()
                sampled_chars.append(marginal_char)
        
        chars = ""
        for i in range(len(sampled_chars)):
            c = sampled_chars[i]
            chars +=  captchaModel.char_dict[c]
        correct = 0
        
        for p_char, t_char in zip(chars, label):
            if p_char == t_char:
                correct += 1
        noise_difference += abs(float(noise) - float(gt_noise))
        if not verbose:
            print("N_predicted:", int(N), "| Actual N:", len(label), "| Predicted Noise:", round(float(noise), 3), "| Actual Noise:", round(float(gt_noise), 3), "| Predicted Text:", chars, "| Actual Text:", label, "| Correct:", correct)
        if correct == len(label) and int(N) == len(label):
            total_correct += 1
        char_correct += correct
        total_char += len(label)
    num_test_samples = i_batch + 1
    accuracy = total_correct / num_test_samples
    char_accuracy = char_correct / total_char
    noise_difference = noise_difference / num_test_samples
    print("use_train =", use_train, "AVG Noise Difference:", noise_difference, "Total correct:", total_correct, "accuracy:{}/{}=".format(total_correct, num_test_samples), accuracy, "char_accuracy:{}/{}=".format(char_correct, total_char), char_accuracy)


In [12]:
def test_cycle(use_cuda):
    
    # disable dropout
    #captchaModel.numNet.eval()
    #captchaModel.charNetSingle.eval()
    test(use_train=True, verbose=True, use_cuda=use_cuda)
    test(1000, use_train=False, verbose=True, use_cuda=use_cuda)
    #test(10, use_train=True, verbose=False, use_cuda=use_cuda)
    test(10, use_train=False, verbose=False, use_cuda=use_cuda)
    # enable dropout
    #captchaModel.numNet.train()
    #captchaModel.charNetSingle.train()

def optimize(start_epoch=1, use_cuda=False):
    """
    Training/Inferencing Stage
    """
    loss_sequence = []
    pause = 5
    save_pause = 10
    print("Optimizing...")
    for t in range(start_epoch, num_steps + 1):
        L = inference(t, use_cuda)
        loss_sequence.append(L)
        if (t % pause == 0) and (t > 0):
            test_cycle(use_cuda=use_cuda)
        if (t % save_pause == 0) and (t > 0):
            save_and_download_checkpoints("branches-1-no-var-no-tanh_model.pt", "branches-1-no-var-no-tanh_optim.pt", "branches-1-no-var-no-tanh_param_store.pt")
    plt.plot(loss_sequence)
    plt.title("loss")
    plt.show()

In [13]:
# saves the model and optimizer states to disk
def save_checkpoint(currentModel, currentOptimzier, save_model, save_opt, save_param_store):
    print("saving model to %s..." % save_model)
    torch.save(currentModel.state_dict(), save_model)
    print("saving optimizer states to %s..." % save_opt)
    currentOptimzier.save(save_opt)
    print("saving pyro pram store states to %s..." % save_param_store)
    pyro.get_param_store().save(save_param_store)
    print("done saving checkpoints to disk.")

# loads the model and optimizer states from disk
def load_checkpoint(myModel, myOptimzer, load_model, load_opt, load_param_store):
    pyro.clear_param_store()
    print("loading model from %s..." % load_model)
    myModel.load_state_dict(torch.load(load_model))
    print("loading optimizer states from %s..." % load_opt)
    myOptimzer.load(load_opt)
    print("loading pyro pram store states from %s..." % load_param_store)
    pyro.get_param_store().load(load_param_store)
    print("done loading states.")
    pyro.module("guide", myModel, update_module_params=True)

def save_and_download_checkpoints(save_model, save_opt, save_param_store):
    save_checkpoint(captchaModel, optimiser, save_model, save_opt, save_param_store)

In [14]:
class CaptchaModel(nn.Module):
    """
    network, model and guide wrapper class
    """
    def __init__(self, use_cuda=False):
        super().__init__()
        self.num_char_domain = torch.arange(MIN_N, MAX_N + 1)
        if use_cuda:
            self.num_char_domain = self.num_char_domain.cuda()

        self.numNet = NumNet((captchaHeight, captchaWidth), len(self.num_char_domain))
        self.noiseNet = NoiseNet((captchaHeight, captchaWidth), 1)
        self.char_dict = char_dict # letter dictionary
        self.rnn_hidden_size = 512
        self.rnn_num_layer = 2
        self.charNet = CharNetSingle((captchaHeight, captchaWidth), len(self.char_dict), max(self.num_char_domain))
        self.noise_constraint = torch.distributions.constraints.interval(MIN_NOISE, MAX_NOISE)
        if use_cuda:
            self.cuda()
        self.use_cuda = use_cuda
    
    def _map_to_noise_range(self, input):
        """
        map input number to the valid noise range
        """
        input = torch.distributions.transform_to(self.noise_constraint)(input)
        return input

    def guide(self, observations={"captcha": torch.rand(1, captchaHeight, captchaWidth)}):
        pyro.module("guide", self)
        img = observations["captcha"].float()
        
        # posterior to the number of letters
        prob = self.numNet(img)
        prob = torch.mean(prob, dim=0)
        N_index = pyro.sample("num_char", dist.Categorical(prob))
        N_index = torch.add(N_index, self.num_char_domain[0])
        
        with pyro.plate("data", img.shape[0]):
            
            # posterior to the noise
            noise_mean, noise_sig, noise_map = self.noiseNet(img)
            noise_batch = pyro.sample("noise", dist.Normal(noise_mean, noise_sig).to_event(1))
            noise_batch = self._map_to_noise_range(noise_batch)
            
            for i in range(N_index):
                charP_i = self.charNet(img, i, noise_map)
                pyro.sample("char_{}".format(i), dist.Categorical(charP_i).to_event(0))
    
    def model(self, observations={"captcha": torch.rand(1, captchaHeight, captchaWidth)}):
        
        BS = observations["captcha"].shape[0]
        
        num_p = torch.tensor(1 / len(self.num_char_domain)).repeat(len(self.num_char_domain))
        
        if self.use_cuda:
            num_p = num_p.cuda()
        
        # sample the number of characters
        N_index = pyro.sample("num_char", dist.Categorical(num_p))
        N_index = torch.add(N_index,  self.num_char_domain[0])
        
        with pyro.plate("data", BS):
            
            noise_mean = torch.tensor((MAX_NOISE - MIN_NOISE) / 2).repeat((BS, 1))
            noise_sig = torch.tensor(0.5).repeat((BS, 1))

            if self.use_cuda:
                noise_mean = noise_mean.cuda()
                noise_sig = noise_sig.cuda()

            # sample the noise
            noise_batch = pyro.sample("noise", dist.Normal(noise_mean, noise_sig).to_event(1))
            noise_batch = self._map_to_noise_range(noise_batch)
            
            sampled_c = []
            for i in range(int(N_index)):
                num_c_i = torch.tensor(1 / len(self.char_dict)).repeat((BS, len(self.char_dict)))
                if self.use_cuda:
                    num_c_i = num_c_i.cuda()
                c_i = pyro.sample("char_{}".format(i), dist.Categorical(num_c_i).to_event(0))
                sampled_c.append(c_i)
            
            # sample characters
            rendered_images = []
            for i in range(sampled_c[0].shape[0]):
                chars = ""
                for j in range(N_index):
                    chars += self.char_dict[sampled_c[j][i]]
        
                rendered_image = render_image(chars, noise=float(noise_batch[i]), use_cuda=self.use_cuda)
                rendered_images.append(rendered_image)
                
        rendered_images = torch.stack(rendered_images)
        sigma = torch.tensor(0.000001)
        if self.use_cuda:
                sigma = sigma.cuda()

        pyro.sample("captcha", dist.Normal(rendered_images, sigma).to_event(2), obs=observations["captcha"])

captchaModel = CaptchaModel(USE_CUDA)

optimiser = pyro.optim.Adam({'lr': 5e-5})
csis = pyro.infer.CSIS(captchaModel.model, captchaModel.guide, optimiser, num_inference_samples=1)


optimize(1, USE_CUDA)
test_cycle(USE_CUDA)

Optimizing...
loss at epoch 1 is 384602421597726.2
loss at epoch 2 is 115936711219661.89
loss at epoch 3 is 74578008164584.62
loss at epoch 4 is 52703317903120.83
loss at epoch 5 is 47238492778555.016
use_train = True AVG Noise Difference: 0.036190565837183275 Total correct: 6265 accuracy:6265/10000= 0.6265 char_accuracy:35654/40153= 0.8879535775658107
use_train = False AVG Noise Difference: 0.03659073389729798 Total correct: 640 accuracy:640/1000= 0.64 char_accuracy:3554/3971= 0.8949886678418535
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.898 | Actual Noise: 0.964 | Predicted Text: 6635 | Actual Text: 6655 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.876 | Actual Noise: 0.976 | Predicted Text: 420 | Actual Text: 420 | Correct: 3
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.866 | Actual Noise: 0.889 | Predicted Text: 94632 | Actual Text: 94632 | Correct: 5
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.859 | Actual Noise: 0.888 | Predicted Text: 9377 | A

done saving checkpoints to disk.
loss at epoch 21 is 24054032267031.355
loss at epoch 22 is 24804937107426.543
loss at epoch 23 is 22582808824315.867
loss at epoch 24 is 23653296961382.008
loss at epoch 25 is 23770622294635.277
use_train = True AVG Noise Difference: 0.028389160814599638 Total correct: 9124 accuracy:9124/10000= 0.9124 char_accuracy:39240/40153= 0.9772619729534531
use_train = False AVG Noise Difference: 0.02899067939577088 Total correct: 903 accuracy:903/1000= 0.903 char_accuracy:3916/4016= 0.9750996015936255
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.732 | Actual Noise: 0.735 | Predicted Text: 497 | Actual Text: 497 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.912 | Actual Noise: 0.97 | Predicted Text: 104 | Actual Text: 104 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.387 | Actual Noise: 0.372 | Predicted Text: 451 | Actual Text: 451 | Correct: 3
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.224 | Actual Noise: 0.206 | Pre

done saving checkpoints to disk.
loss at epoch 41 is 20617876237398.836
loss at epoch 42 is 20136269441446.72
loss at epoch 43 is 19839268932773.473
loss at epoch 44 is 20640590472386.977
loss at epoch 45 is 20143620735401.973
use_train = True AVG Noise Difference: 0.026815302550521794 Total correct: 9406 accuracy:9406/10000= 0.9406 char_accuracy:39540/40153= 0.9847333947650238
use_train = False AVG Noise Difference: 0.02785933482967463 Total correct: 948 accuracy:948/1000= 0.948 char_accuracy:3938/3993= 0.9862258953168044
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.449 | Actual Noise: 0.445 | Predicted Text: 76241 | Actual Text: 76241 | Correct: 5
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.869 | Actual Noise: 0.861 | Predicted Text: 88813 | Actual Text: 88813 | Correct: 5
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.385 | Actual Noise: 0.379 | Predicted Text: 398 | Actual Text: 398 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.639 | Actual Noise: 0.6

done saving checkpoints to disk.
loss at epoch 61 is 18731551909701.24
loss at epoch 62 is 18392134858794.6
loss at epoch 63 is 18520336186382.74
loss at epoch 64 is 18869960354706.223
loss at epoch 65 is 18686857280179.39
use_train = True AVG Noise Difference: 0.025471220836572744 Total correct: 9521 accuracy:9521/10000= 0.9521 char_accuracy:39660/40153= 0.9877219634896521
use_train = False AVG Noise Difference: 0.0239190462596666 Total correct: 948 accuracy:948/1000= 0.948 char_accuracy:3975/4030= 0.9863523573200993
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.203 | Actual Noise: 0.165 | Predicted Text: 69685 | Actual Text: 69685 | Correct: 5
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.686 | Actual Noise: 0.689 | Predicted Text: 50147 | Actual Text: 50147 | Correct: 5
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.201 | Actual Noise: 0.175 | Predicted Text: 71988 | Actual Text: 71988 | Correct: 5
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.166 | Actual Noise: 0.09

done saving checkpoints to disk.
loss at epoch 81 is 18023871049865.598
loss at epoch 82 is 17783121774477.305
loss at epoch 83 is 17992048470845.04
loss at epoch 84 is 17867742105278.867
loss at epoch 85 is 18033811611621.785
use_train = True AVG Noise Difference: 0.022157616458516327 Total correct: 9661 accuracy:9661/10000= 0.9661 char_accuracy:39800/40153= 0.9912086270017184
use_train = False AVG Noise Difference: 0.02184383110474845 Total correct: 967 accuracy:967/1000= 0.967 char_accuracy:3925/3960= 0.9911616161616161
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.936 | Actual Noise: 0.986 | Predicted Text: 1982 | Actual Text: 1982 | Correct: 4
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.835 | Actual Noise: 0.875 | Predicted Text: 3430 | Actual Text: 3430 | Correct: 4
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.21 | Actual Noise: 0.206 | Predicted Text: 099 | Actual Text: 099 | Correct: 3
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.735 | Actual Noise: 0.738 | 

done saving checkpoints to disk.
loss at epoch 101 is 17186714111456.049
loss at epoch 102 is 17271516037926.086
loss at epoch 103 is 16973456022405.137
loss at epoch 104 is 17382468856302.814
loss at epoch 105 is 17359197512518.861
use_train = True AVG Noise Difference: 0.022345273991156295 Total correct: 9699 accuracy:9699/10000= 0.9699 char_accuracy:39844/40153= 0.9923044355340821
use_train = False AVG Noise Difference: 0.023511232726109957 Total correct: 965 accuracy:965/1000= 0.965 char_accuracy:3942/3979= 0.9907011812013069
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.162 | Actual Noise: 0.094 | Predicted Text: 934 | Actual Text: 934 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.647 | Actual Noise: 0.654 | Predicted Text: 5120 | Actual Text: 5120 | Correct: 4
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.298 | Actual Noise: 0.3 | Predicted Text: 66892 | Actual Text: 66892 | Correct: 5
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.571 | Actual Noise: 

done saving checkpoints to disk.
loss at epoch 121 is 16881204226424.832
loss at epoch 122 is 16986322454675.479
loss at epoch 123 is 16936781753443.945
loss at epoch 124 is 16783471322208.67
loss at epoch 125 is 16658188060085.418
use_train = True AVG Noise Difference: 0.02187142770512952 Total correct: 9765 accuracy:9765/10000= 0.9765 char_accuracy:39911/40153= 0.9939730530719996
use_train = False AVG Noise Difference: 0.020905875949800774 Total correct: 976 accuracy:976/1000= 0.976 char_accuracy:4030/4054= 0.9940799210656142
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.225 | Actual Noise: 0.232 | Predicted Text: 4118 | Actual Text: 4118 | Correct: 4
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.696 | Actual Noise: 0.691 | Predicted Text: 59454 | Actual Text: 59454 | Correct: 5
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.386 | Actual Noise: 0.394 | Predicted Text: 72705 | Actual Text: 72705 | Correct: 5
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.807 | Actual Noi

done saving checkpoints to disk.
loss at epoch 141 is 16715244015203.13
loss at epoch 142 is 16442332027558.31
loss at epoch 143 is 16400843381879.613
loss at epoch 144 is 16447482383571.348
loss at epoch 145 is 16552654180699.322
use_train = True AVG Noise Difference: 0.021393281253690253 Total correct: 9751 accuracy:9751/10000= 0.9751 char_accuracy:39899/40153= 0.9936741961995368
use_train = False AVG Noise Difference: 0.02205312139291376 Total correct: 981 accuracy:981/1000= 0.981 char_accuracy:3985/4005= 0.9950062421972534
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.678 | Actual Noise: 0.667 | Predicted Text: 719 | Actual Text: 719 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.912 | Actual Noise: 0.939 | Predicted Text: 2075 | Actual Text: 2775 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.862 | Actual Noise: 0.889 | Predicted Text: 015 | Actual Text: 015 | Correct: 3
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.66 | Actual Noise: 0.649 

KeyboardInterrupt: 